In [1]:
!pip install nltk
import nltk
import pandas as pd
nltk.download("stopwords")
nltk.download('punkt')
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfTransformer


import string
import re


     |████████████████████████████████| 1.4 MB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 719 kB 5.2 MB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434672 sha256=8ce9ae7a4a19ab05639c97c021a26a4bdfb8c0651542b180a1f006414a881e3d
  Stored in directory: /home/jovyan/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
def clean(arg):
    arg=arg.lower()
    arg=arg.replace("_", " ")
    arg=re.sub("","",arg)
    # no sonderzeichen 
    arg=re.sub("(\\d|\\W)+"," ",arg)
    return arg

def sort_tfidf(mat):
    tuples = zip(mat.col, mat.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def topn(feature_names, sorted_items, topn=10):
    #https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [3]:
df = pd.read_csv("args.tsv", sep="\t")

In [4]:
%%time
arg = df["premises"].apply(lambda arg: clean(str(arg)))

CPU times: user 2min 55s, sys: 574 ms, total: 2min 55s
Wall time: 2min 55s


In [5]:
%%time
cv = CountVectorizer(strip_accents = 'unicode', # works 
                                stop_words = set(stopwords.words('english')),
                                lowercase = True, # works
                                max_df = 0.20) # works
word_count_vector=cv.fit_transform(arg)

CPU times: user 1min 38s, sys: 4.64 s, total: 1min 42s
Wall time: 1min 42s


In [6]:
%%time
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

CPU times: user 258 ms, sys: 73.1 ms, total: 332 ms
Wall time: 330 ms


TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [7]:
%%time
feature_names=cv.get_feature_names()
kw = []
for doc in arg:
    tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
    sorted_items=sort_tfidf(tf_idf_vector.tocoo())
    keywords=topn(feature_names,sorted_items,10)
    kw.append(list(keywords.keys()))



CPU times: user 45min 2s, sys: 1.34 s, total: 45min 4s
Wall time: 45min 16s


In [8]:
count = [len(k) for k in kw]
df["keywords"] = kw
df["key_count"] = count

In [9]:
synonyms =[[list(set([i.lemmas()[0].name() for i in wordnet.synsets(word)])) for word in keywords] for keywords in kw]


In [10]:
syn = []

for s in synonyms:
    s.append([" "])
    syn.append(list(set.union(*[set(x) for x in s])))
count = [len(set(s)) for s in syn]
syn = [" ".join(set(s)) for s in syn]

In [11]:

df["synonyms"] = [s.replace("_", "") for s in syn]
df["syn_count"] = count
df.to_csv("cleaned_data_with_key_syn.csv", "\t", index=False)
df["args_naiveexpansion"] = df["premises"] + " " + df["synonyms"]
df.to_csv("naiveexpansion.csv", sep="\t", index=False)

In [12]:
from statistics import mean, stdev

print(mean(count))
print(stdev(count))
kwc = [len(k) for k in kw]
print(mean(kwc))
print(stdev(kwc))

25.30331872682372
11.6967759049616
8.798103658710394
2.5495787965967183
